Created by Joan-Marc Fisa

- Numerai: [FisaGol](https://numer.ai/fisagol)

- Twitter: [@fisagol](https://twitter.com/fisagol)


In [1]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [2]:
!pip install numerapi
!pip install vecstack;

  Created wheel for vecstack: filename=vecstack-0.4.0-py3-none-any.whl size=19878 sha256=7a39da491b12040f48ad0602923b08fa2e3440e270eebe92281a0aab5983f0e4
  Stored in directory: /root/.cache/pip/wheels/28/fe/0c/fe8e43660e3316d7ce204e59a79a72246c0ae9b6c5c79841c8
Successfully built vecstack


In [12]:
import os
import gc
import csv
import sys
import glob
import time
from pathlib import Path
from multiprocessing import Pool

import numerapi

import scipy
import numpy as np
import pandas as pd
import tensorflow as tf
import random
import sklearn
from sklearn import (
    feature_extraction, feature_selection, decomposition, linear_model,
    model_selection, metrics, svm, preprocessing, utils
)
from sklearn.preprocessing import StandardScaler,MinMaxScaler, OrdinalEncoder, LabelEncoder,OneHotEncoder
from keras.models import Sequential, model_from_json, load_model
from keras.layers import Dense, Dropout, Activation,LSTM,Bidirectional, MaxPooling2D, Flatten,GRU
from sklearn.model_selection import StratifiedKFold, GroupKFold, GridSearchCV,cross_val_score,KFold, RepeatedStratifiedKFold,train_test_split
from sklearn.metrics import log_loss, make_scorer, mean_squared_error,classification_report,accuracy_score
from keras.utils import np_utils 
from sklearn import preprocessing
from xgboost import XGBRegressor 
from sklearn.cluster import KMeans
import matplotlib as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler

import math
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, BaggingRegressor
from sklearn.linear_model import SGDRegressor
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor 

import torch.nn as nn
import torch.nn.functional as F

from vecstack import stacking

from sklearn import metrics




In [13]:
'''
##################### CLASSES AND FUNCTIONS ##################################
'''
def my_loss_fn(y_true, y_pred):
    squared_difference = tf.square(y_true - y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)
    
def Cluster_Kmeans(df,X,clusters):
    
    print('Call algorithm K-means')
    kmeans = KMeans(n_clusters=clusters, random_state=rand, init = 'random')
    print('Fitting algorithm K-means')
    kmeans.fit(X)
    print('Finished and Fitted')
    X_clusters_kmeans = df
    X_clusters_kmeans['k-means'] = kmeans.labels_
    return X_clusters_kmeans

def KMeans_Clustering_XGBRegressor(self,df,group,features):
  
    df_features_X = group[features]
    df_features_X['target'] = group.target
    X = df_features_X[df_features_X.columns[0:-1]]
    Y = df_features_X[df_features_X.columns[-1]]
    x_train,x_test,y_train,y_test = train_test_split(X,Y, test_size=0.20,random_state=rand)

    model = XGBRegressor(max_depth=5, learning_rate=0.01, n_estimators=2000, colsample_bytree=0.1, verbosity=1, nthread=6)
    model.fit(x_train, y_train)

    return model, x_train,x_test,y_train,y_test

def standardize_data(df):
    scaler = RobustScaler()
    data = scaler.fit_transform(df)
    return data
#https://joedorfman.github.io/2018/numerai/

def my_loss_fn_1(y_true, y_pred):
    loss = tf.keras.losses.poisson(y_true, y_pred)
    return loss


def my_loss_fn_2(y_true, y_pred):
    threshold = 1     
    error = y_true - y_pred     
    is_small_error = tf.abs(error) <= threshold     
    small_error_loss = tf.square(error) / 2     
    big_error_loss = threshold * (tf.abs(error) - (0.5 * threshold))
    return tf.where(is_small_error, small_error_loss, big_error_loss)

def my_loss_fn_3(y_true, y_pred):
    squared_difference = tf.square(y_true - y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)


def rank_correlation(y_true, y_pred):
    return np.nan_to_num(stats.spearmanr(y_true, y_pred, axis=1)[0], nan=-1)


def rmsle_1(y_true, y_pred):
    return 'RMSLE', np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2))), False

def rae_1(y_true, y_pred):
    return 'RAE', np.sum(np.abs(y_pred - y_true)) / np.sum(np.abs(np.mean(y_true) - y_true)), False


def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5


#https://github.com/microsoft/LightGBM/blob/master/examples/python-guide/sklearn_example.py
#https://github.com/chricke/Numerai-Keras/tree/master
#https://forum.numer.ai/t/example-for-using-tensorflow-keras-with-a-custom-callback/855


'''
y_pred=[…]
y_true=[…]
corr_coefs=[…] #310 correlation coefficients with y_pred

mse= mean(square(y_pred - y_true)
correlation_penalty= square(std(abs(corr_coefs)))
loss= mse + correlation_penalty
'''

'\ny_pred=[…]\ny_true=[…]\ncorr_coefs=[…] #310 correlation coefficients with y_pred\n\nmse= mean(square(y_pred - y_true)\ncorrelation_penalty= square(std(abs(corr_coefs)))\nloss= mse + correlation_penalty\n'

In [14]:
TOURNAMENT_NAME = "nomi"
TARGET_NAME = f"target"
PREDICTION_NAME = f"prediction"

BENCHMARK = 0
BAND = 0.2

In [15]:
##############################################################################
########################## DOWLOAD DATA ######################################
##############################################################################

In [16]:
seed_1 = 1
rand_1 = np.random.seed(seed_1)
seed_2 = rand_1
rand_2 = np.random.seed(seed_2)
seed_3 = rand_2
rand_3 = np.random.seed(seed_3)

In [17]:
napi = numerapi.NumerAPI(verbosity="info")

napi.download_current_dataset(unzip=True)

current_ds = napi.get_current_round()
latest_round = os.path.join('numerai_dataset_'+str(current_ds))

2021-08-28 19:00:12,673 INFO numerapi.utils: target file already exists
2021-08-28 19:00:12,674 INFO numerapi.utils: download complete
2021-08-28 19:00:12,677 INFO numerapi.base_api: unzipping file...


In [18]:
##################################################################
##################### LOAD DATA ##################################
##################################################################

In [19]:
%%time
print("# Loading data...")

training_data = pd.read_csv(os.path.join(latest_round, "numerai_training_data.csv")).set_index("id")
tournament_data = pd.read_csv(os.path.join(latest_round, "numerai_tournament_data.csv")).set_index("id")
validation_data = tournament_data[tournament_data.data_type == "validation"]

print("# All Loaded...")

# Loading data...
# All Loaded...
CPU times: user 1min 11s, sys: 11.5 s, total: 1min 23s
Wall time: 1min 22s


In [20]:
feature_names = [f for f in training_data.columns if f.startswith("feature")]
print(f"Loaded {len(feature_names)} features")

Loaded 310 features


In [21]:
X = training_data[feature_names]
Y = training_data[training_data.columns[-1]]

le = LabelEncoder()
Y_enc = le.fit_transform(Y)

In [22]:
%%time
print('Call algorithm K-means')
kmeans = KMeans(n_clusters=10, random_state=rand_1, init = 'random')
print('Fitting algorithm K-means')
kmeans.fit(X)
print('Finished and Fitted')
X_clusters_kmeans = training_data
X_clusters_kmeans['k-means'] = kmeans.labels_

Call algorithm K-means
Fitting algorithm K-means
Finished and Fitted
CPU times: user 10min 50s, sys: 4.01 s, total: 10min 54s
Wall time: 10min 49s


In [23]:
training_data["id_seq"] = list(range(len(training_data)))
tournament_data["id_seq"] = list(range(len(tournament_data)))

In [24]:
tournament_data

,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,feature_intelligence8,feature_intelligence9,feature_intelligence10,feature_intelligence11,feature_intelligence12,feature_charisma1,feature_charisma2,feature_charisma3,feature_charisma4,feature_charisma5,feature_charisma6,feature_charisma7,feature_charisma8,feature_charisma9,feature_charisma10,feature_charisma11,feature_charisma12,feature_charisma13,feature_charisma14,feature_charisma15,feature_charisma16,feature_charisma17,feature_charisma18,feature_charisma19,feature_charisma20,feature_charisma21,feature_charisma22,feature_charisma23,feature_charisma24,feature_charisma25,feature_charisma26,...,feature_wisdom9,feature_wisdom10,feature_wisdom11,feature_wisdom12,feature_wisdom13,feature_wisdom14,feature_wisdom15,feature_wisdom16,feature_wisdom17,feature_wisdom18,feature_wisdom19,feature_wisdom20,feature_wisdom21,feature_wisdom22,feature_wisdom23,feature_wisdom24,feature_wisdom25,feature_wisdom26,feature_wisdom27,feature_wisdom28,feature_wisdom29,feature_wisdom30,feature_wisdom31,feature_wisdom32,feature_wisdom33,feature_wisdom34,feature_wisdom35,feature_wisdom36,feature_wisdom37,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target,id_seq
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
n0003aa52cab36c2,era121,validation,0.25,0.75,0.50,0.50,0.00,0.75,0.50,0.25,0.50,0.50,0.25,0.00,0.25,0.50,0.25,0.00,0.25,1.00,1.00,0.25,1.00,1.00,0.25,0.25,0.00,0.50,0.25,0.75,0.00,0.50,0.25,0.25,0.25,0.50,0.00,0.50,1.00,0.25,...,0.00,0.25,0.5,0.25,0.25,0.00,0.25,0.00,0.25,0.50,0.50,0.50,0.50,0.00,0.25,0.75,0.25,0.25,0.50,0.25,0.00,0.25,0.50,0.25,0.5,0.25,0.25,1.00,0.75,0.75,0.75,1.00,0.75,0.50,0.5,1.00,0.00,0.00,0.25,0
n000920ed083903f,era121,validation,0.75,0.50,0.75,1.00,0.50,0.00,0.00,0.75,0.25,0.00,0.75,0.50,0.00,0.25,0.50,0.00,1.00,0.25,0.25,1.00,1.00,0.25,0.75,0.00,0.00,0.75,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.25,0.00,1.00,0.25,...,0.50,0.25,1.0,0.50,0.25,0.00,0.25,0.50,0.25,1.00,0.25,0.00,0.50,0.75,0.75,0.50,1.00,1.00,0.25,0.50,0.25,0.50,0.50,0.50,0.5,0.25,0.25,0.75,0.50,0.50,0.50,0.75,1.00,0.75,0.5,0.50,0.50,0.50,0.50,1
n0038e640522c4a6,era121,validation,1.00,0.00,0.00,1.00,1.00,1.00,1.00,1.00,0.50,0.50,1.00,1.00,1.00,0.75,0.50,0.50,1.00,1.00,0.50,0.50,0.00,1.00,0.50,1.00,0.50,1.00,0.50,1.00,0.25,1.00,1.00,1.00,0.50,1.00,1.00,0.75,1.00,1.00,...,0.50,0.00,0.0,0.00,0.25,0.25,0.00,0.50,0.00,0.00,0.00,0.25,0.00,0.25,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,0.75,0.0,0.00,0.25,0.00,0.00,0.00,0.00,0.50,0.25,0.00,0.0,0.50,0.50,0.00,1.00,2
n004ac94a87dc54b,era121,validation,0.75,1.00,1.00,0.50,0.00,0.00,0.00,0.50,0.75,1.00,0.75,0.00,0.50,0.00,0.50,0.75,0.50,0.75,0.25,0.75,0.25,0.75,0.25,0.75,1.00,0.50,0.50,0.75,0.50,1.00,0.50,0.25,0.75,0.25,0.75,0.25,0.75,0.75,...,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.25,0.00,0.25,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.75,0.00,0.00,0.25,0.25,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.0,0.00,0.25,0.25,0.50,3
n0052fe97ea0c05f,era121,validation,0.25,0.50,0.50,0.25,1.00,0.50,0.50,0.25,0.25,0.50,0.50,1.00,1.00,1.00,1.00,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.00,0.25,0.00,0.00,0.75,0.25,1.00,0.25,1.00,0.75,0.00,1.00,0.75,0.75,0.75,0.25,...,0.50,0.50,0.0,0.75,0.50,0.75,0.25,0.25,0.25,0.00,0.25,0.50,0.25,1.00,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.75,1.00,1.0,0.75,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.75,1.0,0.00,0.25,1.00,0.75,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nffd1cda0c61ecdb,eraX,live,0.50,0.50,0.50,0

In [25]:
X_group_1 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 0]
X_group_2 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 1]
X_group_3 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 2]
X_group_4 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 3]
X_group_5 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 4]
X_group_6 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 5]
X_group_7 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 6]
X_group_8 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 7]
X_group_9 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 8]
X_group_10 = X_clusters_kmeans.loc[X_clusters_kmeans['k-means'] == 9]

gc.collect()

2021-08-28 19:12:48,956 INFO numexpr.utils: Note: NumExpr detected 40 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-08-28 19:12:48,957 INFO numexpr.utils: NumExpr defaulting to 8 threads.


0

In [26]:
def Grouping_Models(df_train):
  
    df_train_all = df_train[feature_names]
    df_train_all[TARGET_NAME] = df_train['target'].values
    X = df_train_all[df_train_all.columns[0:-1]]
    Y = df_train_all[df_train_all.columns[-1]]
    x_train,x_test,y_train,y_test = train_test_split(X,Y, test_size=0.20,random_state=rand_3)
    model = LGBMRegressor(random_state=rand_1)
    model.fit(x_train, y_train, eval_set=[(x_test, y_test)])
    
    return model

In [27]:
model_group_1 = Grouping_Models(X_group_1)
gc.collect()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


[1]	valid_0's l2: 0.0680179
[2]	valid_0's l2: 0.0680207
[3]	valid_0's l2: 0.0680017
[4]	valid_0's l2: 0.0679893
[5]	valid_0's l2: 0.0680163
[6]	valid_0's l2: 0.0680029
[7]	valid_0's l2: 0.068003
[8]	valid_0's l2: 0.0680113
[9]	valid_0's l2: 0.0680183
[10]	valid_0's l2: 0.0680272
[11]	valid_0's l2: 0.0680051
[12]	valid_0's l2: 0.0680086
[13]	valid_0's l2: 0.068012
[14]	valid_0's l2: 0.0680042
[15]	valid_0's l2: 0.0680063
[16]	valid_0's l2: 0.0680069
[17]	valid_0's l2: 0.0680189
[18]	valid_0's l2: 0.0680209
[19]	valid_0's l2: 0.0680376
[20]	valid_0's l2: 0.0680666
[21]	valid_0's l2: 0.0680593
[22]	valid_0's l2: 0.0680647
[23]	valid_0's l2: 0.0680778
[24]	valid_0's l2: 0.0681005
[25]	valid_0's l2: 0.0681138
[26]	valid_0's l2: 0.0681094
[27]	valid_0's l2: 0.0681015
[28]	valid_0's l2: 0.0681188
[29]	valid_0's l2: 0.0681395
[30]	valid_0's l2: 0.0681605
[31]	valid_0's l2: 0.0681752
[32]	valid_0's l2: 0.0681736
[33]	valid_0's l2: 0.0681658
[34]	valid_0's l2: 0.0681638
[35]	valid_0's l2: 0.0681

236

In [28]:
model_group_2 = Grouping_Models(X_group_2)
gc.collect()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


[1]	valid_0's l2: 0.0321674
[2]	valid_0's l2: 0.0321683
[3]	valid_0's l2: 0.0321582
[4]	valid_0's l2: 0.0321495
[5]	valid_0's l2: 0.0321483
[6]	valid_0's l2: 0.0321412
[7]	valid_0's l2: 0.032137
[8]	valid_0's l2: 0.0321439
[9]	valid_0's l2: 0.0321493
[10]	valid_0's l2: 0.0321414
[11]	valid_0's l2: 0.0321435
[12]	valid_0's l2: 0.032133
[13]	valid_0's l2: 0.0321317
[14]	valid_0's l2: 0.0321376
[15]	valid_0's l2: 0.0321266
[16]	valid_0's l2: 0.0321251
[17]	valid_0's l2: 0.0321272
[18]	valid_0's l2: 0.0321275
[19]	valid_0's l2: 0.0321313
[20]	valid_0's l2: 0.0321303
[21]	valid_0's l2: 0.0321203
[22]	valid_0's l2: 0.0321163
[23]	valid_0's l2: 0.0321126
[24]	valid_0's l2: 0.0321089
[25]	valid_0's l2: 0.0321159
[26]	valid_0's l2: 0.0321187
[27]	valid_0's l2: 0.032121
[28]	valid_0's l2: 0.0321266
[29]	valid_0's l2: 0.0321234
[30]	valid_0's l2: 0.0321226
[31]	valid_0's l2: 0.0321218
[32]	valid_0's l2: 0.0321296
[33]	valid_0's l2: 0.0321268
[34]	valid_0's l2: 0.0321271
[35]	valid_0's l2: 0.03212

236

In [29]:
model_group_3 = Grouping_Models(X_group_3)
gc.collect()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


[1]	valid_0's l2: 0.0480779
[2]	valid_0's l2: 0.048072
[3]	valid_0's l2: 0.0480625
[4]	valid_0's l2: 0.0480635
[5]	valid_0's l2: 0.0480696
[6]	valid_0's l2: 0.0480554
[7]	valid_0's l2: 0.0480579
[8]	valid_0's l2: 0.0480512
[9]	valid_0's l2: 0.0480569
[10]	valid_0's l2: 0.0480437
[11]	valid_0's l2: 0.0480494
[12]	valid_0's l2: 0.0480388
[13]	valid_0's l2: 0.0480378
[14]	valid_0's l2: 0.0480333
[15]	valid_0's l2: 0.0480361
[16]	valid_0's l2: 0.0480339
[17]	valid_0's l2: 0.0480351
[18]	valid_0's l2: 0.0480186
[19]	valid_0's l2: 0.0480209
[20]	valid_0's l2: 0.0480278
[21]	valid_0's l2: 0.0480356
[22]	valid_0's l2: 0.0480251
[23]	valid_0's l2: 0.0480349
[24]	valid_0's l2: 0.0480473
[25]	valid_0's l2: 0.0480488
[26]	valid_0's l2: 0.0480494
[27]	valid_0's l2: 0.0480387
[28]	valid_0's l2: 0.0480392
[29]	valid_0's l2: 0.0480378
[30]	valid_0's l2: 0.0480445
[31]	valid_0's l2: 0.0480593
[32]	valid_0's l2: 0.0480794
[33]	valid_0's l2: 0.0480739
[34]	valid_0's l2: 0.0480955
[35]	valid_0's l2: 0.048

236

In [30]:
model_group_4 = Grouping_Models(X_group_4)
gc.collect()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


[1]	valid_0's l2: 0.0456579
[2]	valid_0's l2: 0.0456717
[3]	valid_0's l2: 0.0456746
[4]	valid_0's l2: 0.0456798
[5]	valid_0's l2: 0.0456722
[6]	valid_0's l2: 0.0456748
[7]	valid_0's l2: 0.0456802
[8]	valid_0's l2: 0.0456842
[9]	valid_0's l2: 0.0456898
[10]	valid_0's l2: 0.0457039
[11]	valid_0's l2: 0.0457052
[12]	valid_0's l2: 0.0457093
[13]	valid_0's l2: 0.0457153
[14]	valid_0's l2: 0.0457183
[15]	valid_0's l2: 0.0457231
[16]	valid_0's l2: 0.0457315
[17]	valid_0's l2: 0.0457393
[18]	valid_0's l2: 0.045746
[19]	valid_0's l2: 0.0457415
[20]	valid_0's l2: 0.0457392
[21]	valid_0's l2: 0.0457515
[22]	valid_0's l2: 0.0457645
[23]	valid_0's l2: 0.0457679
[24]	valid_0's l2: 0.0457794
[25]	valid_0's l2: 0.0457785
[26]	valid_0's l2: 0.0457819
[27]	valid_0's l2: 0.0457907
[28]	valid_0's l2: 0.0458093
[29]	valid_0's l2: 0.0458221
[30]	valid_0's l2: 0.0458251
[31]	valid_0's l2: 0.0458184
[32]	valid_0's l2: 0.0458022
[33]	valid_0's l2: 0.0457984
[34]	valid_0's l2: 0.0458056
[35]	valid_0's l2: 0.045

236

In [31]:
model_group_5 = Grouping_Models(X_group_5)
gc.collect()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


[1]	valid_0's l2: 0.0590433
[2]	valid_0's l2: 0.0590396
[3]	valid_0's l2: 0.0590325
[4]	valid_0's l2: 0.059039
[5]	valid_0's l2: 0.0590432
[6]	valid_0's l2: 0.0590424
[7]	valid_0's l2: 0.0590578
[8]	valid_0's l2: 0.0590731
[9]	valid_0's l2: 0.0590917
[10]	valid_0's l2: 0.0590908
[11]	valid_0's l2: 0.0591025
[12]	valid_0's l2: 0.0590812
[13]	valid_0's l2: 0.0590958
[14]	valid_0's l2: 0.0590753
[15]	valid_0's l2: 0.0590775
[16]	valid_0's l2: 0.0591132
[17]	valid_0's l2: 0.0591109
[18]	valid_0's l2: 0.0591178
[19]	valid_0's l2: 0.0591299
[20]	valid_0's l2: 0.0591478
[21]	valid_0's l2: 0.0591604
[22]	valid_0's l2: 0.0591678
[23]	valid_0's l2: 0.059176
[24]	valid_0's l2: 0.0591852
[25]	valid_0's l2: 0.0591936
[26]	valid_0's l2: 0.0592159
[27]	valid_0's l2: 0.0592273
[28]	valid_0's l2: 0.0592327
[29]	valid_0's l2: 0.0592309
[30]	valid_0's l2: 0.059238
[31]	valid_0's l2: 0.0592484
[32]	valid_0's l2: 0.0592784
[33]	valid_0's l2: 0.0592609
[34]	valid_0's l2: 0.0592619
[35]	valid_0's l2: 0.05926

216

In [32]:
model_group_6 = Grouping_Models(X_group_6)
gc.collect()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


[1]	valid_0's l2: 0.0298471
[2]	valid_0's l2: 0.0298475
[3]	valid_0's l2: 0.0298428
[4]	valid_0's l2: 0.0298437
[5]	valid_0's l2: 0.0298463
[6]	valid_0's l2: 0.0298435
[7]	valid_0's l2: 0.0298458
[8]	valid_0's l2: 0.0298402
[9]	valid_0's l2: 0.0298412
[10]	valid_0's l2: 0.0298509
[11]	valid_0's l2: 0.0298479
[12]	valid_0's l2: 0.0298555
[13]	valid_0's l2: 0.0298572
[14]	valid_0's l2: 0.0298521
[15]	valid_0's l2: 0.0298548
[16]	valid_0's l2: 0.0298561
[17]	valid_0's l2: 0.0298743
[18]	valid_0's l2: 0.0298631
[19]	valid_0's l2: 0.0298731
[20]	valid_0's l2: 0.0298769
[21]	valid_0's l2: 0.0298706
[22]	valid_0's l2: 0.0298637
[23]	valid_0's l2: 0.0298623
[24]	valid_0's l2: 0.0298589
[25]	valid_0's l2: 0.0298599
[26]	valid_0's l2: 0.0298585
[27]	valid_0's l2: 0.0298547
[28]	valid_0's l2: 0.0298583
[29]	valid_0's l2: 0.0298678
[30]	valid_0's l2: 0.0298748
[31]	valid_0's l2: 0.0298669
[32]	valid_0's l2: 0.029869
[33]	valid_0's l2: 0.0298742
[34]	valid_0's l2: 0.0298732
[35]	valid_0's l2: 0.029

236

In [33]:
model_group_7 = Grouping_Models(X_group_7)
gc.collect()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


[1]	valid_0's l2: 0.0541279
[2]	valid_0's l2: 0.0541152
[3]	valid_0's l2: 0.0541165
[4]	valid_0's l2: 0.0541133
[5]	valid_0's l2: 0.0541267
[6]	valid_0's l2: 0.0541462
[7]	valid_0's l2: 0.0541595
[8]	valid_0's l2: 0.0541631
[9]	valid_0's l2: 0.0541786
[10]	valid_0's l2: 0.0541908
[11]	valid_0's l2: 0.0541929
[12]	valid_0's l2: 0.0542028
[13]	valid_0's l2: 0.0541915
[14]	valid_0's l2: 0.0542034
[15]	valid_0's l2: 0.0541973
[16]	valid_0's l2: 0.0542058
[17]	valid_0's l2: 0.0541776
[18]	valid_0's l2: 0.0541806
[19]	valid_0's l2: 0.0541957
[20]	valid_0's l2: 0.0542074
[21]	valid_0's l2: 0.0542091
[22]	valid_0's l2: 0.0542009
[23]	valid_0's l2: 0.0542115
[24]	valid_0's l2: 0.0542227
[25]	valid_0's l2: 0.0542211
[26]	valid_0's l2: 0.0542332
[27]	valid_0's l2: 0.0542293
[28]	valid_0's l2: 0.0542177
[29]	valid_0's l2: 0.0542118
[30]	valid_0's l2: 0.0542231
[31]	valid_0's l2: 0.0542323
[32]	valid_0's l2: 0.054257
[33]	valid_0's l2: 0.0542508
[34]	valid_0's l2: 0.0542675
[35]	valid_0's l2: 0.054

236

In [34]:
model_group_8 = Grouping_Models(X_group_8)
gc.collect()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


[1]	valid_0's l2: 0.0457287
[2]	valid_0's l2: 0.0457505
[3]	valid_0's l2: 0.0457615
[4]	valid_0's l2: 0.0457612
[5]	valid_0's l2: 0.0457532
[6]	valid_0's l2: 0.0457786
[7]	valid_0's l2: 0.0457843
[8]	valid_0's l2: 0.0458032
[9]	valid_0's l2: 0.0458087
[10]	valid_0's l2: 0.0457983
[11]	valid_0's l2: 0.0457887
[12]	valid_0's l2: 0.0457935
[13]	valid_0's l2: 0.0458164
[14]	valid_0's l2: 0.0458114
[15]	valid_0's l2: 0.0458219
[16]	valid_0's l2: 0.0458373
[17]	valid_0's l2: 0.0458425
[18]	valid_0's l2: 0.0458415
[19]	valid_0's l2: 0.0458507
[20]	valid_0's l2: 0.0458763
[21]	valid_0's l2: 0.0458724
[22]	valid_0's l2: 0.0458649
[23]	valid_0's l2: 0.0458523
[24]	valid_0's l2: 0.0458546
[25]	valid_0's l2: 0.0458579
[26]	valid_0's l2: 0.0458525
[27]	valid_0's l2: 0.045859
[28]	valid_0's l2: 0.0458598
[29]	valid_0's l2: 0.0458837
[30]	valid_0's l2: 0.0458922
[31]	valid_0's l2: 0.0458976
[32]	valid_0's l2: 0.0459029
[33]	valid_0's l2: 0.0459108
[34]	valid_0's l2: 0.0459095
[35]	valid_0's l2: 0.045

236

In [35]:
model_group_9 = Grouping_Models(X_group_9)
gc.collect()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


[1]	valid_0's l2: 0.057151
[2]	valid_0's l2: 0.0571282
[3]	valid_0's l2: 0.0571194
[4]	valid_0's l2: 0.0571047
[5]	valid_0's l2: 0.057104
[6]	valid_0's l2: 0.0570958
[7]	valid_0's l2: 0.0570958
[8]	valid_0's l2: 0.0570977
[9]	valid_0's l2: 0.0571003
[10]	valid_0's l2: 0.0570939
[11]	valid_0's l2: 0.0571084
[12]	valid_0's l2: 0.0571085
[13]	valid_0's l2: 0.057107
[14]	valid_0's l2: 0.0571189
[15]	valid_0's l2: 0.0571091
[16]	valid_0's l2: 0.0570988
[17]	valid_0's l2: 0.0570964
[18]	valid_0's l2: 0.0570955
[19]	valid_0's l2: 0.0570823
[20]	valid_0's l2: 0.0571028
[21]	valid_0's l2: 0.0570918
[22]	valid_0's l2: 0.0570984
[23]	valid_0's l2: 0.057103
[24]	valid_0's l2: 0.0570946
[25]	valid_0's l2: 0.0570922
[26]	valid_0's l2: 0.0571091
[27]	valid_0's l2: 0.0570982
[28]	valid_0's l2: 0.0570937
[29]	valid_0's l2: 0.0571003
[30]	valid_0's l2: 0.0571054
[31]	valid_0's l2: 0.0571071
[32]	valid_0's l2: 0.0571163
[33]	valid_0's l2: 0.0571064
[34]	valid_0's l2: 0.0571167
[35]	valid_0's l2: 0.057118

236

In [36]:
model_group_10 = Grouping_Models(X_group_10)
gc.collect()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


[1]	valid_0's l2: 0.075214
[2]	valid_0's l2: 0.0752608
[3]	valid_0's l2: 0.0752348
[4]	valid_0's l2: 0.0752468
[5]	valid_0's l2: 0.0752673
[6]	valid_0's l2: 0.0752996
[7]	valid_0's l2: 0.0752973
[8]	valid_0's l2: 0.075332
[9]	valid_0's l2: 0.0753091
[10]	valid_0's l2: 0.0753595
[11]	valid_0's l2: 0.0753369
[12]	valid_0's l2: 0.0753475
[13]	valid_0's l2: 0.0753574
[14]	valid_0's l2: 0.0753635
[15]	valid_0's l2: 0.0753513
[16]	valid_0's l2: 0.0753418
[17]	valid_0's l2: 0.0753486
[18]	valid_0's l2: 0.0753761
[19]	valid_0's l2: 0.0753624
[20]	valid_0's l2: 0.0753821
[21]	valid_0's l2: 0.0753937
[22]	valid_0's l2: 0.075406
[23]	valid_0's l2: 0.0754166
[24]	valid_0's l2: 0.0754229
[25]	valid_0's l2: 0.0754498
[26]	valid_0's l2: 0.0754347
[27]	valid_0's l2: 0.0754372
[28]	valid_0's l2: 0.0754969
[29]	valid_0's l2: 0.0755037
[30]	valid_0's l2: 0.0755344
[31]	valid_0's l2: 0.0755429
[32]	valid_0's l2: 0.0755592
[33]	valid_0's l2: 0.0755556
[34]	valid_0's l2: 0.0755847
[35]	valid_0's l2: 0.07558

236

In [37]:
########################## PREDICTIONS ON LEBELS ##############################

In [38]:
labels_tournament = kmeans.predict(tournament_data[feature_names])
tournament_data['k-means'] = labels_tournament
gc.collect()

100

In [39]:
X_group_1_td = tournament_data.loc[tournament_data['k-means'] == 0]
X_group_2_td = tournament_data.loc[tournament_data['k-means'] == 1]
X_group_3_td = tournament_data.loc[tournament_data['k-means'] == 2]
X_group_4_td = tournament_data.loc[tournament_data['k-means'] == 3]
X_group_5_td = tournament_data.loc[tournament_data['k-means'] == 4]
X_group_6_td = tournament_data.loc[tournament_data['k-means'] == 5]
X_group_7_td = tournament_data.loc[tournament_data['k-means'] == 6]
X_group_8_td = tournament_data.loc[tournament_data['k-means'] == 7]
X_group_9_td = tournament_data.loc[tournament_data['k-means'] == 8]
X_group_10_td = tournament_data.loc[tournament_data['k-means'] == 9]
gc.collect()

0

In [40]:
prediction_group_1 = pd.DataFrame()
prediction_group_1[PREDICTION_NAME] = model_group_1.predict(X_group_1_td[feature_names])
prediction_group_1["id_seq"] = X_group_1_td["id_seq"].values
gc.collect()

54

In [41]:
prediction_group_2 = pd.DataFrame()
prediction_group_2[PREDICTION_NAME] = model_group_2.predict(X_group_2_td[feature_names])
prediction_group_2["id_seq"] = X_group_2_td["id_seq"].values
gc.collect()

54

In [42]:
prediction_group_3 = pd.DataFrame()
prediction_group_3[PREDICTION_NAME] = model_group_3.predict(X_group_3_td[feature_names])
prediction_group_3["id_seq"] = X_group_3_td["id_seq"].values
gc.collect()

54

In [43]:
prediction_group_4 = pd.DataFrame()
prediction_group_4[PREDICTION_NAME] = model_group_4.predict(X_group_4_td[feature_names])
prediction_group_4["id_seq"] = X_group_4_td["id_seq"].values
gc.collect()

54

In [44]:
prediction_group_5 = pd.DataFrame()
prediction_group_5[PREDICTION_NAME] = model_group_5.predict(X_group_5_td[feature_names])
prediction_group_5["id_seq"] = X_group_5_td["id_seq"].values
gc.collect()

54

In [45]:
prediction_group_6 = pd.DataFrame()
prediction_group_6[PREDICTION_NAME] = model_group_6.predict(X_group_6_td[feature_names])
prediction_group_6["id_seq"] = X_group_6_td["id_seq"].values
gc.collect()

56

In [46]:
prediction_group_7 = pd.DataFrame()
prediction_group_7[PREDICTION_NAME] = model_group_7.predict(X_group_7_td[feature_names])
prediction_group_7["id_seq"] = X_group_7_td["id_seq"].values
gc.collect()

54

In [47]:
prediction_group_8 = pd.DataFrame()
prediction_group_8[PREDICTION_NAME] = model_group_8.predict(X_group_8_td[feature_names])
prediction_group_8["id_seq"] = X_group_8_td["id_seq"].values
gc.collect()

54

In [48]:
prediction_group_9 = pd.DataFrame()
prediction_group_9[PREDICTION_NAME] = model_group_9.predict(X_group_9_td[feature_names])
prediction_group_9["id_seq"] = X_group_9_td["id_seq"].values
gc.collect()

54

In [49]:
prediction_group_10 = pd.DataFrame()
prediction_group_10[PREDICTION_NAME] = model_group_10.predict(X_group_10_td[feature_names])
prediction_group_10["id_seq"] = X_group_10_td["id_seq"].values
gc.collect()

54

In [50]:
########################## UNION OF PREDICTIONS ##############################

In [51]:
vertical_stack = pd.concat([prediction_group_1, prediction_group_2,prediction_group_3,prediction_group_4,prediction_group_5
                            ,prediction_group_6,prediction_group_7,prediction_group_8,prediction_group_9
                            ,prediction_group_10], axis=0)
vertical_stack = vertical_stack.sort_values("id_seq")
gc.collect()

100

In [52]:
tournament_data[PREDICTION_NAME] = vertical_stack[PREDICTION_NAME].values
gc.collect()

50

In [53]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.483362
n000920ed083903f    0.509107
n0038e640522c4a6    0.509851
n004ac94a87dc54b    0.519979
n0052fe97ea0c05f    0.496464
                      ...   
nffd1cda0c61ecdb    0.508712
nffd711b0e6007e3    0.464781
nffd8a4d389df16a    0.499018
nffdec2abd3d4c1e    0.522369
nfff8751e4f86103    0.485397
Name: prediction, Length: 1800934, dtype: float64

In [54]:
def neutralize(df, columns, by, proportion=1.0):
  
    scores = df[columns]
    exposures = df[by].values
    # constant column to make sure the series is completely neutral to exposures
    exposures = np.hstack((exposures, np.array([np.mean(scores)] * len(exposures)).reshape(-1, 1)))
    gc.collect()
    scores = scores - proportion * exposures.dot(np.linalg.pinv(exposures).dot(scores))
    gc.collect()
    return scores / scores.std()


In [55]:
%%time
tournament_data[PREDICTION_NAME] = neutralize(tournament_data, PREDICTION_NAME, feature_names)

CPU times: user 10min 7s, sys: 10min 40s, total: 20min 47s
Wall time: 1min 4s


In [56]:
def minmax_norm(df):
    return (df - df.min()) / (df.max() - df.min())

In [57]:
tournament_data[PREDICTION_NAME] = minmax_norm(tournament_data[PREDICTION_NAME])

In [58]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.491729
n000920ed083903f    0.547250
n0038e640522c4a6    0.487436
n004ac94a87dc54b    0.567322
n0052fe97ea0c05f    0.496939
                      ...   
nffd1cda0c61ecdb    0.514370
nffd711b0e6007e3    0.406846
nffd8a4d389df16a    0.496299
nffdec2abd3d4c1e    0.524558
nfff8751e4f86103    0.453076
Name: prediction, Length: 1800934, dtype: float64

In [59]:
##############################################################################
######################### MAKE SUBMISSION #################################### 
##############################################################################

In [60]:
tournament_data[PREDICTION_NAME].to_csv("BeeChain_7.csv")

In [62]:
!cp BeeChain_7.csv "drive/My Drive/BeeChain Foundation/"